In [1]:
#finds site map of entire domain
from usp.tree import sitemap_tree_for_homepage

#for saving and loading objects
import pickle

#for accessing url html
import requests
import urllib.request, urllib.parse, urllib.error
from urllib.request import Request, urlopen

#for html parsing
from bs4 import BeautifulSoup

#anxillary functions/methods
import numpy as np
import pandas as pd
import time
import random
import datetime
from dateutil.tz import tzutc, tzoffset

pd.set_option('display.max_colwidth', -1)
from tqdm import tqdm, trange
import time

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


# Sitemap

In [ ]:
#SOUTH CHINA MORNING POST
# scmp articles as of date 4/11/2020. 
#tree = sitemap_tree_for_homepage('https://www.scmp.com/hk');

In [ ]:
urls = []
dates = []
for page in tree.all_pages():
    urls.append(page.url)
    dates.append(page.last_modified)

In [ ]:
!ls ../Pickles

In [ ]:
#transpose
zippedList =list(zip(urls,dates))

#format into dataframe
df = pd.DataFrame(zippedList, columns = ['urls' ,'date'])

#drop nan for date published
df = df.dropna()


# Data Scrape

In [6]:
#LOAD DATASET OF ALL SCRAPED URLS FROM SCMP
with open('../Pickles/SCMP_All.pkl', 'rb') as f:
    df = pickle.load(f)
    
#reset df index
df = df.reset_index(drop = True)

#find articles with "Articles" to exclude photos/video pages
articles_only_urls = []
for i in trange(len(df)-1):
    if 'article' in df.urls[i] and 'hong-kong' in df.urls[i] and df.date[i].year == 2019:
        articles_only_urls.append(df.urls[i])


100%|██████████| 1413330/1413330 [01:21<00:00, 17324.32it/s]


In [10]:
author = []
title = []
date = []
summary = []
link = []
topic = []
broken = []

In [11]:
#using BS4 to request each html data
for i in tqdm(articles_only_urls):
    try:
        time.sleep(1) 
        req = Request(str(i), headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'lxml')

        by = soup.find("meta", {"name": "cse_author"})
        ti = soup.find("meta", {"name": "cse_headline"})
        to = soup.find("meta", {"name": "cse_topic"})
        d = soup.find("meta", {"property": "article:published_time"})
        summ = soup.find("meta", {"name": "cse_summary"})


        if by and ti and d:
            author.append(by["content"])
            title.append(ti["content"])
            topic.append(to["content"])
            date.append(d["content"])
            summary.append(summ["content"])
            link.append(i)

            if len(link)% 1000 == 0:
                print("Dataset contains " + str(len(link)) + " entries.")
    except:
        print("URL number " + str(articles_only_urls.index(i)) + " is broken: " + i )
        broken.append(i)
        pass


  9%|▉         | 2015/22172 [1:11:19<7:59:19,  1.43s/it] 

Dataset contains 1000 entries.


 10%|█         | 2327/22172 [1:23:44<8:23:35,  1.52s/it] 

URL number 2326 is broken: https://www.scmp.com/comment/letters/article/3034473/want-better-future-hong-kong-start-ending-chaos-legislative-council


 14%|█▎        | 3017/22172 [1:46:50<11:56:44,  2.25s/it]

...

 94%|█████████▍| 20846/22172 [9:50:22<32:59,  1.49s/it]  

Dataset contains 19000 entries.


 99%|█████████▊| 21876/22172 [10:17:18<08:13,  1.67s/it] 

Dataset contains 20000 entries.


100%|██████████| 22172/22172 [10:25:09<00:00,  1.69s/it]


In [12]:
df = pd.DataFrame(list(zip(title, author, date, summary, topic,link)), 
               columns =['Title', 'Author', 'Date', 'Summary','Topic','Link']) 

#df.to_pickle("./SCMP_Opinions.pkl")
df

,Title,Author,Date,Summary,Topic,Link
0,Hong Kong protesters and mainland gamers clash in Grand Theft Auto V Online,Josh Ye,2019-12-23T12:45:34+08:00,,"Hong Kong protests,Gaming,Steam,Propaganda,Abacus - China Tech News",https://www.scmp.com/abacus/games/article/3043211/hong-kong-protesters-and-mainland-gamers-clash-grand-theft-auto-v
1,China has its own Hong Kong protest game that lets you beat up activists,Josh Ye,2019-12-06T10:00:24+08:00,"After protesters turned to video game activism, a new web game lets players beat figures like Joshua Wong and Martin Lee with a flip-flop or baseball bat","Gaming,Hong Kong protests,Steam,Abacus - China Tech News",https://www.scmp.com/abacus/games/article/3040864/china-has-its-own-hong-kong-protest-game-lets-you-beat-activists
2,Tim Cook says Apple made decision to remove Hong Kong protest app “unilaterally”,Karen Chiu,2019-11-21T16:27:58+08:00,Apple CEO talks about business in China during a visit from Donald Trump to new Texas facility,"Apple,iPhone,Hong Kong protests,Apps,Cybersecurity,Abacus - China Tech News",https://www.scmp.com/abacus/culture/article/3038759/tim-cook-says-apple-made-decision-remove-hong-kong-protest-app
3,Hong Kong bike sharing firm unlocks its bikes amid protests,Karen Chiu,2019-11-12T19:47:20+08:00,LocoBike says its shared bikes are free for students to take due to “system breakdown”,"Sharing economy,Apps,Hong Kong protests,Abacus - China Tech News",https://www.scmp.com/abacus/news-bites/article/3037426/hong-kong-bike-sharing-firm-unlocks-its-bikes-amid-protests
4,A new Steam game lets you fight as a Hong Kong protester,Josh Ye,2019-11-09T10:00:13+08:00,"Liberate Hong Kong was inspired by Blizzard’s Blitzchung controversy, and now it’s making Steam the next battleground for protesters","Steam,Hong Kong protests,Gaming,Abacus - China Tech News",https://www.scmp.com/abacus/games/article/3037019/new-steam-game-lets-you-fight-hong-kong-protester
...,...,...,...,...,...,...
20284,Hong Kong’s nano drone racers find space to fly in concrete jungle,Karen Chiu,2018-05-15T19:15:45+08:00,Finding space to fly a drone among Hong Kong's densely packed skyscrapers isn't easy. But that hasn't stopped the city's tight-knit community of racing enthusiasts. Watch our video to see what it's like to race through an indoor playground -- through the eyes of a drone.,"Drones,Abacus - China Tech News",https://www.scmp.com/abacus/tech/article/3028503/hong-kongs-nano-drone-racers-find-space-fly-concrete-jungle
20285,We tested a messaging app used by Hong Kong protesters that works without an internet connection,Masha Borak,2019-09-10T19:00:06+08:00,Bridgefy is a Bluetooth mesh network app trending in Hong Kong,"Trending,Apps,Hong Kong protests,Abacus - China Tech News",https://www.scmp.com/abacus/culture/article/3026527/we-tested-messaging-app-used-hong-kong-protesters-works-without
20286,Why are Hong Kong protesters targeting lamp posts?,Masha Borak,2019-08-30T09:33:06+08:00,Protesters tore down a smart lamp post and posted online what they found inside,"Surveillance,Trending,Facial recognition,Privacy,Hong Kong extradition bill,Hong Kong protests,Abacus - China Tech News",https://www.scmp.com/abacus/culture/article/3024997/why-are-hong-kong-protesters-targeting-lamp-posts
20287,Securing smartphones is now a priority for Hong Kong protesters,Masha Borak,2019-08-08T13:16:52+08:00,,"Cybersecurity,Trending,Abacus - China Tech News",https://www.scmp.com/abacus/culture/article/3021937/securing-smartphones-now-priority-hong-kong-protesters


In [13]:
with open('../Pickles/ScmpDF.pkl', 'wb') as f:
    pickle.dump(df, f)

# Data

In [14]:
with open('../Pickles/ScmpDF.pkl', 'rb') as f:
    df = pickle.load(f)


# Test Set

In [15]:
df

,Title,Author,Date,Summary,Topic,Link
0,Hong Kong protesters and mainland gamers clash in Grand Theft Auto V Online,Josh Ye,2019-12-23T12:45:34+08:00,,"Hong Kong protests,Gaming,Steam,Propaganda,Abacus - China Tech News",https://www.scmp.com/abacus/games/article/3043211/hong-kong-protesters-and-mainland-gamers-clash-grand-theft-auto-v
1,China has its own Hong Kong protest game that lets you beat up activists,Josh Ye,2019-12-06T10:00:24+08:00,"After protesters turned to video game activism, a new web game lets players beat figures like Joshua Wong and Martin Lee with a flip-flop or baseball bat","Gaming,Hong Kong protests,Steam,Abacus - China Tech News",https://www.scmp.com/abacus/games/article/3040864/china-has-its-own-hong-kong-protest-game-lets-you-beat-activists
2,Tim Cook says Apple made decision to remove Hong Kong protest app “unilaterally”,Karen Chiu,2019-11-21T16:27:58+08:00,Apple CEO talks about business in China during a visit from Donald Trump to new Texas facility,"Apple,iPhone,Hong Kong protests,Apps,Cybersecurity,Abacus - China Tech News",https://www.scmp.com/abacus/culture/article/3038759/tim-cook-says-apple-made-decision-remove-hong-kong-protest-app
3,Hong Kong bike sharing firm unlocks its bikes amid protests,Karen Chiu,2019-11-12T19:47:20+08:00,LocoBike says its shared bikes are free for students to take due to “system breakdown”,"Sharing economy,Apps,Hong Kong protests,Abacus - China Tech News",https://www.scmp.com/abacus/news-bites/article/3037426/hong-kong-bike-sharing-firm-unlocks-its-bikes-amid-protests
4,A new Steam game lets you fight as a Hong Kong protester,Josh Ye,2019-11-09T10:00:13+08:00,"Liberate Hong Kong was inspired by Blizzard’s Blitzchung controversy, and now it’s making Steam the next battleground for protesters","Steam,Hong Kong protests,Gaming,Abacus - China Tech News",https://www.scmp.com/abacus/games/article/3037019/new-steam-game-lets-you-fight-hong-kong-protester
...,...,...,...,...,...,...
20284,Hong Kong’s nano drone racers find space to fly in concrete jungle,Karen Chiu,2018-05-15T19:15:45+08:00,Finding space to fly a drone among Hong Kong's densely packed skyscrapers isn't easy. But that hasn't stopped the city's tight-knit community of racing enthusiasts. Watch our video to see what it's like to race through an indoor playground -- through the eyes of a drone.,"Drones,Abacus - China Tech News",https://www.scmp.com/abacus/tech/article/3028503/hong-kongs-nano-drone-racers-find-space-fly-concrete-jungle
20285,We tested a messaging app used by Hong Kong protesters that works without an internet connection,Masha Borak,2019-09-10T19:00:06+08:00,Bridgefy is a Bluetooth mesh network app trending in Hong Kong,"Trending,Apps,Hong Kong protests,Abacus - China Tech News",https://www.scmp.com/abacus/culture/article/3026527/we-tested-messaging-app-used-hong-kong-protesters-works-without
20286,Why are Hong Kong protesters targeting lamp posts?,Masha Borak,2019-08-30T09:33:06+08:00,Protesters tore down a smart lamp post and posted online what they found inside,"Surveillance,Trending,Facial recognition,Privacy,Hong Kong extradition bill,Hong Kong protests,Abacus - China Tech News",https://www.scmp.com/abacus/culture/article/3024997/why-are-hong-kong-protesters-targeting-lamp-posts
20287,Securing smartphones is now a priority for Hong Kong protesters,Masha Borak,2019-08-08T13:16:52+08:00,,"Cybersecurity,Trending,Abacus - China Tech News",https://www.scmp.com/abacus/culture/article/3021937/securing-smartphones-now-priority-hong-kong-protesters
